# Cassiopeia ecDNA simulations

Here, we simulate a population of cells with extrachromosomal DNA (ecDNA), circular, acentromeric DNA molecules that arise in cancers and segregate randomly at mitosis. This tutorial is based on benchmark.ipynb and we quote / borrow extensively from it. 

"You can install Cassiopeia by following the guide [here](https://cassiopeia-lineage.readthedocs.io/en/latest/installation.html)."

"All of our documentation is hosted [here](https://cassiopeia-lineage.readthedocs.io/en/latest/)."

In [1]:
import cProfile
from collections import defaultdict
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import time
from tqdm.auto import tqdm

import cassiopeia as cas
from cassiopeia.solver import missing_data_methods

SyntaxError: positional argument follows keyword argument (ecDNABirthDeathSimulator.py, line 401)

In [ ]:
from IPython.display import Image


## Tree Simulation

ecDNABirthDeathSimulator is a subclass of BirthDeathFitnesSimulator, which is documented [here](https://cassiopeia-lineage.readthedocs.io/en/latest/api/reference/cassiopeia.sim.BirthDeathFitnessSimulator.html). 

In [ ]:
bd_sim = cas.sim.ecDNABirthDeathSimulator(
    birth_waiting_distribution = lambda scale: np.random.exponential(scale),
    initial_birth_scale = 0.5,
    death_waiting_distribution = lambda: np.inf, # lambda: np.random.exponential(1.5),
    # mutation_distribution = lambda: 1 if np.random.uniform() < 0.5 else 0,
    # fitness_distribution = lambda: np.random.normal(0, .5),
    # fitness_base = 1.3,
    num_extant = 4,
    random_seed=17, 
    initial_copy_number = np.array([7,3,5]),
    splitting_function = lambda x: np.random.binomial(x, p=0.5),
    fitness_array = np.array([[[0,0.1], [0.1,0.2]],[[0.1,0.5],[0.2,0.6]]])
)
ground_truth_tree = bd_sim.simulate_tree()

We can manually inspect the tree (when the number of cells is not too large) to see how random segregation leads to asymmetric CN. 

In [ ]:
# ground_truth_tree.__dict__  
ground_truth_tree._CassiopeiaTree__network.__dict__

In [ ]:
for node in ground_truth_tree.get_tree_topology().nodes:
    print(node)

Unfortunately, iTOL is no longer free. However, there is local plotting functionality. For tutorials, see Cassiopeia/notebooks/local_plotting.ipynb

In [ ]:
cas.pl.plot_matplotlib(ground_truth_tree, add_root=True)

Now, let's annotate the tree with the copy numbers.

## Overlay Lineage Tracing Data

We can then use our Cas9-based lineage tracing data simulator to overlay data. Here, this is a very flexible Cas9-data generator in which we can specify the dependence structure between characters (i.e. the cassettes that link togehter cut sites), the mutation rate, silencing rates, and state distributions. This function derives from the DataSimulator class & implements the `overlay_data` function which operates on a `CassiopeiaTree`, such as the one you simulated above. 

In [ ]:
np.random.seed(seed=None)
lt_sim = cas.sim.Cas9LineageTracingDataSimulator(
    number_of_cassettes = 40,
    size_of_cassette = 1,
    mutation_rate = 0.1,
    state_generating_distribution = lambda: np.random.exponential(1e-5),
    number_of_states = 50,
    state_priors = None,
    heritable_silencing_rate = 9e-4,
    stochastic_silencing_rate = 0.1,
    heritable_missing_data_state = -1,
    stochastic_missing_data_state = -1,
)
lt_sim.overlay_data(ground_truth_tree)

We can get a feel for the character matrix and the complexity of the data here by extracting the simulated data.

In [ ]:
## inspect character matrix

character_matrix = ground_truth_tree.character_matrix

print(f"We simulated {character_matrix.shape[1]} characters across {character_matrix.shape[0]} cells.")

# plot the number of unique states per character
character_matrix.nunique(axis=0).plot(kind='bar')
plt.xlabel("Character")
plt.ylabel("Number of unique states ")
plt.show()

missing_data_per_character = character_matrix.apply(lambda x: len(x[x == -1]) / len(x), axis=0)
missing_data_per_character.plot(kind='bar')
plt.xlabel("Character")
plt.ylabel("Percentage of missing states")
plt.show()

## Rebuild trees

Cassiopeia has implemented several CassiopeiaSolvers for reconstructing trees. Each of these can take in several class-specific parameters and at a minimum implements the `solve` routine which operates on a CassiopeiaTree. 

The full list of solvers can be found [here](https://cassiopeia-lineage.readthedocs.io/en/latest/api/solver.html). For a full tutorial on tree reconstruction, refer to the [Tree Reconstruction notebook](https://github.com/YosefLab/Cassiopeia/blob/latest/notebooks/reconstruct.ipynb).

Here we use the VanillaGreedySolver, which was described in the [Cassiopeia paper published in 2020](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-020-02000-8).

In [ ]:
reconstructed_tree = cas.data.CassiopeiaTree(character_matrix = ground_truth_tree.character_matrix, missing_state_indicator = -1)

greedy_solver = cas.solver.VanillaGreedySolver()
greedy_solver.solve(reconstructed_tree)


In [ ]:
cas.pl.plot_matplotlib(reconstructed_tree, add_root=True)

## Critique

Our `critique` library is for testing the similarity between CassiopeiaTrees. The most pertinent use case is when comparing the reconstruction accuracy. We have two metrics implemented - a triplets correct measure, and the Robinson-Foulds metric.

In [ ]:
reconstructed_tree.collapse_mutationless_edges(infer_ancestral_characters = True)

rf, rf_max = cas.critique.compare.robinson_foulds(ground_truth_tree, reconstructed_tree)

print(f"Normalized RF: {rf / rf_max}")

In [ ]:
triplets = cas.critique.compare.triplets_correct(ground_truth_tree, reconstructed_tree, number_of_trials = 500)

## Building benchmarking pipelines

We can link all of these steps together to benchmark the performance of several algorithms.

The briefest of benchmarking pipelines would consist of:

- Simulating a ground truth phylogeny
- Overlaying data onto this phylogeny
- Inferring the phylogeny with potentially many algorithms
- Assessing performance

Here we show a very simple benchmarking pipeline:

In [ ]:

algorithms = {"Vanilla Greedy": cas.solver.VanillaGreedySolver(),
              'UPGMA': cas.solver.UPGMASolver(dissimilarity_function=cas.solver.dissimilarity.weighted_hamming_distance),
              'NeighborJoining': cas.solver.NeighborJoiningSolver(dissimilarity_function=cas.solver.dissimilarity.weighted_hamming_distance, add_root=True)
             }

iterations = 10

algorithm_to_performance_triplets = defaultdict(list)
algorithm_to_performance_rf = defaultdict(list)
for _ in tqdm(range(iterations)):
    
    ground_truth_tree = bd_sim.simulate_tree()
    
    np.random.seed(None)
    lt_sim.overlay_data(ground_truth_tree)
    
    for algorithm_name in tqdm(algorithms.keys()):
        
        algorithm = algorithms[algorithm_name]
        
        reconstructed_tree = cas.data.CassiopeiaTree(character_matrix = ground_truth_tree.character_matrix, missing_state_indicator = -1)
        algorithm.solve(reconstructed_tree)
        
        # ground_truth_tree.collapse_mutationless_edges(infer_ancestral_characters = False)
        reconstructed_tree.collapse_mutationless_edges(infer_ancestral_characters = True)

        rf, rf_max = cas.critique.compare.robinson_foulds(ground_truth_tree, reconstructed_tree)
        
        triplets = cas.critique.compare.triplets_correct(ground_truth_tree, reconstructed_tree, number_of_trials=500)
        algorithm_to_performance_triplets[algorithm_name].append(np.mean(list(triplets[0].values())))
        
        algorithm_to_performance_rf[algorithm_name].append(rf / rf_max)

        
        
        

We can easily visualize results after reformatting the dictionaries of performances

In [ ]:
algorithm_to_performance_triplets_df = pd.DataFrame(columns = ['Algorithm', 'Triplets'])
for algorithm in algorithm_to_performance_triplets:
    
    entries = algorithm_to_performance_triplets[algorithm]
    new_df = pd.DataFrame([algorithm]*len(entries), columns = ['Algorithm'])
    new_df['Triplets'] = entries
    
    algorithm_to_performance_triplets_df = pd.concat([algorithm_to_performance_triplets_df, new_df])
    
algorithm_to_performance_rf_df = pd.DataFrame(columns = ['Algorithm', 'RF'])
for algorithm in algorithm_to_performance_rf:
    
    entries = algorithm_to_performance_rf[algorithm]
    new_df = pd.DataFrame([algorithm]*len(entries), columns = ['Algorithm'])
    new_df['RF'] = entries
    
    algorithm_to_performance_rf_df = pd.concat([algorithm_to_performance_rf_df, new_df])

In [ ]:
sns.boxplot(data=algorithm_to_performance_triplets_df, x = 'Algorithm', y = 'Triplets')
plt.show()

sns.boxplot(data=algorithm_to_performance_rf_df, x = 'Algorithm', y = 'RF')
plt.show()

In [ ]:
print(algorithm_to_performance_triplets_df)